In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
hipp_train = pd.read_csv('train_radiomics_hipocamp.csv')
hipp_test = pd.read_csv('test_radiomics_hipocamp.csv')
df = pd.read_csv('train_radiomics_hipocamp.csv')

In [ ]:
df.dtypes
df.info()
df.describe()


In [ ]:
df.head()

In [ ]:
df.isna().any()
print(df.isna().sum())

In [ ]:
print(df.duplicated().sum())

In [ ]:
# print(df['Target'].value_counts())
df['diagnostics_Image-original_Mean'].hist()
plt.title('Histogram of diagnostics_Image-original_Mean')
plt.xlabel('Mean Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False ,cbar=False, cmap='viridis')
plt.show()

In [ ]:
sex_counts = df['Sex'].value_counts()
print(sex_counts)

# secalhar o 1 é mulheres e o 0 é homens tenho que ver isto melhor

In [ ]:
#TEMOS DE TORNAR OS OBJETOS EM VALORES NÙMERICOS PARA PUDERMOS UTILIZAR OS MODELOS DE MACHINE LEARNING
from sklearn.preprocessing import OneHotEncoder


categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

#Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(df[categorical_columns])

#Create a DataFrame with the one-hot encoded columns
#We use get_feature_names_out() to get the column names for the encoded data
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))


print(one_hot_df.head())



In [ ]:
df_encoded = pd.concat([df, one_hot_df], axis=1)

# Drop the original categorical columns
df_encoded = df_encoded.drop(categorical_columns, axis=1)

# Display the resulting dataframe
print(f"Encoded data : \n{df_encoded}")

In [ ]:


sns.boxplot(x='diagnostics_Image-original_Maximum', y='Transition', data=df)
plt.show()

print(df)

In [ ]:
contador = 0 
for column in hipp_test.columns:
    if hipp_test[column].nunique() == 1: 
        print(f"A coluna: '{column}' tem apenas um valor único.")
        contador +=1
print(f"Colunas com valor 1:{contador}")

In [ ]:
#Verificar as colunas que tem apenas 1 valor 
contador = 0
for column in hipp_train.columns:
    if hipp_train[column].nunique() == 1: 
        print(f"A coluna: '{column}' tem apenas 1 valor único.")
        contador +=1 
print(f"Numero total de colunas de um valor: '{contador}'")

In [ ]:
hipp_train_c = hipp_train.loc[:, hipp_train.nunique() > 1]
colunas_removidas = hipp_train.columns[hipp_train.nunique() == 1]
print(f"Colunas removidas: {colunas_removidas}")

In [ ]:
#Remover os valores nunique de treino
hipp_test_c = hipp_test.loc[:, hipp_test.nunique() > 1]
colunas_removidas = hipp_test.columns[hipp_test.nunique() == 1]
print(f"As colunas que foram removidas '{colunas_removidas}'")

In [ ]:
mapping = {
    'CN-CN': 1,  # Estado Normal
    'CN-MCI': 2,  # Estado Intermediário
    'MCI-MCI': 2,  # Estado Intermediário
    'MCI-AD': 3,  # Demência
    'AD-AD': 3   # Demência
}

# Aplicar o mapeamento à coluna 'Transition'
hipp_train['Transition'] = hipp_train['Transition'].map(mapping)

# Verificar o resultado
# print(hipp_train[['Transition', 'Transition_Numeric']].head())



In [ ]:
hipp_train.head()

In [ ]:
# Separate features (X) and target labels (y) USAR ISTO PARA O MODELO DE MACHINE LEARNING
X_train = hipp_train.drop(columns=['ID','Label'])  # Replace 'Label' with the actual column name for your target
y_train = hipp_train['Label']                 # Target variable in the training set

X_test = hipp_test.drop(columns=['ID','Label'])    # Features only for testing
y_test = hipp_test['Label']                   # True labels for testing (for evaluation only)

#X_train contains the features (input variables) for training the model.
#y_train contains the target labels (output) that correspond to each row in X_train.
#X_test contains the features for testing how well the model performs on unseen data.
#y_test contains the true labels for the test data, which you use to evaluate the model’s predictions.

In [ ]:
from sklearn.model_selection import train_test_split

#Atençao isto é só para testar o split, depois temos de fazer o split com o dataset todo
colunas_remover = ['Image', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash',
                   'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex',
                   'diagnostics_Mask-original_CenterOfMass', 'Mask']
hipp_train = hipp_train.drop(columns=colunas_remover, errors='ignore').loc[:, hipp_train.nunique() > 1]
hipp_test = hipp_test.drop(columns=colunas_remover, errors='ignore').loc[:, hipp_test.nunique() > 1]


X_train = hipp_train.drop(['Transition', 'ID'], axis=1)
y_train = hipp_train['Transition']

# Dividir o conjunto de treino em treino e teste
X_train_split, X_test, y_train_split, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=2021)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
# Predict on the test set
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# Initialize the SVM classifier
clf = DecisionTreeClassifier(random_state=2021)

# Train the model on the training set
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# could not convert string to float: '/notebooks/disk2/DS2_FreeSurfer/ADNI_006_S_0681_MR_MP-RAGE__br_raw_20060831143335593_1_S18451_I23677_generate/mri/brain.nii.gz
# Evaluate the model's performance
#accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.2f}")

f1 = f1_score(y_test, y_pred, average='weighted')

print("F1 Score:", f1)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
import pandas as pd

# Example predictions
y_pred = ["CN-CN"] * 100  # Replace with actual predictions

# Create a DataFrame with RowId and Result columns
df = pd.DataFrame({
    "RowId": range(1, len(y_pred) + 1),  # RowId starting from 1
    "Result": y_pred                     # Model predictions
})

# Save the DataFrame to a CSV file
df.to_csv("predictions.csv", index=False)

print("Predictions saved to 'predictions.csv'")